In [1]:
!pip install pytorch_lightning wandb

     |████████████████████████████████| 409kB 2.8MB/s 
     |████████████████████████████████| 1.6MB 12.8MB/s 
     |████████████████████████████████| 2.8MB 19.2MB/s 
     |████████████████████████████████| 276kB 34.1MB/s 
     |████████████████████████████████| 829kB 34.5MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 163kB 40.1MB/s 
     |████████████████████████████████| 122kB 42.8MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=9c4bfec62d00f1fd410e0576d1f40b43bfff569b8636f5503212c649c7e5cd1e
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=0d5bbe878432b4fba9b7c844e21f96a8f6c7f31183643e7fc0355107e0fb8441
  Stored in directory:

In [2]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torchtext import data
import numpy as np
import pytorch_lightning as pl

from torchsummary import summary
from pytorch_lightning.loggers import WandbLogger

In [6]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.long,batch_first=True)

train_data=data.TabularDataset(path = "./drive/My Drive/train.txt",csv_reader_params={'delimiter':';'},format = 'csv',fields = [('text',TEXT),('label',LABEL)],skip_header = False)
val_data = data.TabularDataset(path = "./drive/My Drive/val.txt",csv_reader_params={'delimiter':';'},format = 'csv',fields = [('text',TEXT),('label',LABEL)],skip_header = False)
test_data = data.TabularDataset(path = "./drive/My Drive/test.txt",csv_reader_params={'delimiter':';'},format = 'csv',fields = [('text',TEXT),('label',LABEL)],skip_header = False)

train_iter, valid_iter = data.BucketIterator.splits(
    (train_data, val_data), 
    batch_size = 64,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    shuffle = True,
    device = 'cuda' if torch.cuda.is_available() else 'cpu')


In [14]:
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 398911/400000 [00:23<00:00, 19050.73it/s]

Size of TEXT vocabulary: 5226
Size of LABEL vocabulary: 6
[('i', 29007), ('feel', 11183), ('and', 9589), ('to', 8972), ('the', 8370), ('a', 6201), ('that', 5217), ('feeling', 5112), ('of', 4990), ('my', 4283)]
defaultdict(<function _default_unk_index at 0x7f82b65a7598>, {'<unk>': 0, '<pad>': 1, 'i': 2, 'feel': 3, 'and': 4, 'to': 5, 'the': 6, 'a': 7, 'that': 8, 'feeling': 9, 'of': 10, 'my': 11, 'in': 12, 'it': 13, 'like': 14, 'm': 15, 'so': 16, 'for': 17, 'was': 18, 'me': 19, 'have': 20, 'but': 21, 'is': 22, 'this': 23, 'am': 24, 'with': 25, 'not': 26, 'about': 27, 'be': 28, 'nt': 29, 'as': 30, 'you': 31, 'on': 32, 'do': 33, 'just': 34, 'at': 35, 'when': 36, 'or': 37, 'all': 38, 'because': 39, 'more': 40, 'can': 41, 'really': 42, 'up': 43, 'are': 44, 't': 45, 'by': 46, 'very': 47, 'been': 48, 'know': 49, 'if': 50, 'out': 51, 'myself': 52, 'what': 53, 's': 54, 'time': 55, 'how': 56, 'get': 57, 've': 58, 'little': 59, 'had': 60, 'they': 61, 'he': 62, 'now': 63, 'will': 64, 'from': 65, 'wo

In [8]:
class EmotionClassifier(pl.LightningModule):

  def __init__(self, num_embedding, embedding_dim = 100, hidden_size = 512):
    super(EmotionClassifier,self).__init__()

    self.embedding = nn.Embedding(num_embedding, embedding_dim)

    self.lstm = nn.LSTM(embedding_dim, 
                        hidden_size, 
                        num_layers=2, 
                        bidirectional=True, 
                        dropout=0.15,
                        batch_first=True)
    
    self.fc = nn.Linear(hidden_size * 2, 6)

    self.activation = nn.Softmax(dim = 1)

    self.crit = nn.CrossEntropyLoss();

  def forward(self,x):
    text, text_lengths = x.text
    embed = self.embedding(text)
    packed = nn.utils.rnn.pack_padded_sequence(embed,text_lengths,batch_first=True)
    _, (hidden, _) = self.lstm(packed)
    #hidden = [batch size, num layers * num directions,hid dim]
    #cell = [batch size, num layers * num directions,hid dim]
  
    hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

    result = self.fc(hidden)
    return self.activation(result)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer
  
  def training_step(self, batch, batch_idx):
    y_hat = self(batch)
    loss = self.crit(y_hat, batch.label)
    result = pl.TrainResult(loss)
    result.log('train_loss', loss)
    _,idx = torch.max(y_hat,dim=1)
    acc = (batch.label == idx).sum().float()/len(batch.label)
    result.log('train_acc',acc)
    return result

  def validation_step(self, batch, batch_idx):
    y_hat = self(batch)
    loss = self.crit(y_hat, batch.label)
    result = pl.EvalResult(checkpoint_on=loss)
    result.log('val_loss', loss)
    # print(len(batch.label),batch.label.shape,y_hat.shape)
    _,idx = torch.max(y_hat,dim=1)
    acc = (batch.label == idx).sum().float()/len(batch.label)
    result.log('val_acc',acc)
    return result

  def test_step(self, batch, batch_idx):
    y_hat = self(batch)
    loss = self.crit(y_hat, batch.label)
    _,idx = torch.max(y_hat,dim=1)
    acc = (batch.label == idx).sum().float()/len(batch.label)
    result = pl.EvalResult()
    result.log('test_loss', loss)
    result.log('test_acc',acc)
    return result

In [13]:
model = EmotionClassifier(len(TEXT.vocab))

wandb = WandbLogger(project = "Emotion_Classification")
trainer = pl.Trainer(gpus=-1,
                     weights_save_path="./drive/My Drive/EC_save", 
                     max_epochs=100,
                    #  limit_train_batches=0.1,
                    #  limit_val_batches=0.1,
                     logger=wandb,
                     early_stop_callback=True,
                     )
trainer.fit(model,train_iter,valid_iter)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
wandb: Waiting for W&B process to finish, PID 582
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200928_132732-2qkf0722/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200928_132732-2qkf0722/logs/debug-internal.log
wandb: Run summary:
wandb:   global_step 8749
wandb:    train_loss 1.0597378015518188
wandb:     train_acc 0.984375
wandb:         epoch 34
wandb:         _step 209
wandb:      _runtime 606
wandb:    _timestamp 1601300262
wandb:    valid_loss 1.1236865520477295
wandb:     valid_acc 0.9194999933242798
wandb: Run history:
wandb:   global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:    train_loss ██▇▇▆▄▄▃▄▄▂▂▂▂▁▂▂▁▂▁▁▂▂▂▁▂▁▁▁▂▁▁▂▁▁▂▁▁▁▂
wandb:     train_acc ▁▂▂▃▄▅▅▆▆▅▇▇▇▇█▇▇█▇██▇▇▇█▇███▇██▇██▇███▇
wandb:         epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:         _step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      _runtime ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:    _timestamp ▁▁▁▂▂▂▂


  | Name       | Type             | Params
------------------------------------------------
0 | embedding  | Embedding        | 522 K 
1 | lstm       | LSTM             | 8 M   
2 | fc         | Linear           | 6 K   
3 | activation | Softmax          | 0     
4 | crit       | CrossEntropyLoss | 0     


Saving latest checkpoint..


1

In [12]:
trainer=pl.Trainer(resume_from_checkpoint="/content/drive/My Drive/EC_save/Emotion_Classification/1ob1vexv/checkpoints/epoch=77.ckpt",gpus=-1)
model = EmotionClassifier(5226)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [15]:
test_iter = data.BucketIterator(test_data,64,sort_key=lambda x: len(x.text),sort_within_batch=True)
trainer.test(model,test_iter)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9180, device='cuda:0'),
 'test_loss': tensor(1.1244, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_acc': 0.9179999828338623, 'test_loss': 1.1244350671768188}]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

model = EmotionClassifier();
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
state_dict = torch.load('./drive/My Drive/EC_save/EC1 ep acc loss.pt',map_location = 'cuda' if torch.cuda.is_available() else 'cpu');
model.load_state_dict(state_dict)
model.eval();

datal= DataLoader(TensorDataset(test_data,torch.tensor(test_labels)),batch_size=1, pin_memory = True, shuffle = True);
sum = 0;
i = 0;
confusion_matrix = np.zeros((6,6))  # row is truth and col is prediction
with torch.no_grad():
  for i,item in enumerate(datal):
    data = item[0].to('cuda' if torch.cuda.is_available() else 'cpu') 
    label = item[1].to('cuda' if torch.cuda.is_available() else 'cpu')
    output = model(data);
    _,y_pred = torch.max(output,1);
    # print(i,output.tolist(), y_pred.tolist(),label.tolist())
    for k in range(label.shape[0]):
      confusion_matrix[label[k],y_pred[k]] += 1
    sum = sum + (torch.sum(y_pred == label)).item();
    if (i == 500):
      break
  rowsum = np.sum(confusion_matrix,axis = 1);
  rowsum = np.where(rowsum == 0, 1, rowsum)
  confusion_matrix /= rowsum[:,None]
  print("output = ",sum/i);
  from sklearn.metrics import ConfusionMatrixDisplay
  disp = ConfusionMatrixDisplay(confusion_matrix,range(15))
  disp.plot(values_format='.1f')